# Open Pit Mining

In [1]:
import ee
import geemap
import json
import pandas as pd

#ee.Authenticate()
ee.Initialize(project="mining-457808")

In [2]:
with open("C:\\space_technologies\\OpenPitMining\\niedzwiedzia_gora_polygon.geojson") as f:
    geojson_data = json.load(f)

polygon = geemap.geojson_to_ee(geojson_data)

In [24]:
with open("C:\\space_technologies\\OpenPitMining\\niedzwiedzia_gora_buildings.geojson") as f:
    buildings_geojson = json.load(f)

buildings = geemap.geojson_to_ee(buildings_geojson)

In [3]:
date_list_start = pd.date_range(start='2016-01-01', end='2025-03-01', freq='MS')
date_list_start = [date.strftime('%Y-%m-%d') for date in date_list_start]
date_list_end = pd.date_range(start='2016-01-31', end='2025-03-31', freq='M')
date_list_end = [date.strftime('%Y-%m-%d') for date in date_list_end]

C:\Users\Hp\AppData\Local\Temp\ipykernel_11564\3106269892.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_list_end = pd.date_range(start='2016-01-31', end='2025-03-31', freq='M')


### ONE YEAR

In [37]:
image_case = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterBounds(polygon.geometry())
    .filterDate(f'2024-07-01', f'2024-07-31')
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first()
)
image_case

In [14]:
rectangle_case = polygon.geometry().buffer(500).bounds()

bounds = rectangle_case.bounds().getInfo()['coordinates'][0]

# Overpass potrzebuje: (S, W, N, E)
lats = [pt[1] for pt in bounds]
lons = [pt[0] for pt in bounds]

south = min(lats)
north = max(lats)
west = min(lons)
east = max(lons)

print(f"{south}, {west}, {north}, {east}")


50.094972773723526, 19.625012901371132, 50.11341026110655, 19.649379925684382


In [40]:
rectangle_case = polygon.geometry().buffer(500).bounds()

image_case = image_case.clip(rectangle_case)
rgb_case = image_case.select(['B4', 'B3', 'B2'])
ndvi_case = image_case.normalizedDifference(['B8', 'B4'])
bare_soil_case = ndvi_case.lt(0.18)
    

In [58]:

ones_case = ee.Image.constant(1).clip(rectangle_case)
polygon_mask_case = ee.Image.constant(1).clip(polygon)
outside_mask_case = ones_case.where(polygon_mask_case.eq(1), 0)

exceeded_case = bare_soil_case.updateMask(outside_mask_case)

ones_case_b = ee.Image.constant(1).clip(rectangle_case)

buildings_mask = ee.Image().byte().paint(featureCollection=buildings, color=1).clip(rectangle_case)
buildings_mask = ones_case.where(buildings_mask.eq(1), 0)

# buildings_mask = buildings_mask.neq(1)

# Odwracamy maskę: 1 tam, gdzie NIE MA budynków
# non_building_mask = buildings_mask.neq(1)

# Zastosuj maskę — usuń piksele w miejscach budynków
exceeded_case = exceeded_case.updateMask(buildings_mask)


In [59]:

ndvi_vis = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
rgb_vis = {'min': 0, 'max': 3000}
polygon_style = {"color": "red", "fillColor": "00000000", "width": 1}
bare_soil_vis = {'min': 0, 'max': 1, 'palette': ['black', 'white']}
buildings_style = {"color": "green", "fillColor": "00000000", "width": 1}
exceeded_vis = {'min': 0, 'max': 1, 'palette': ['black', 'yellow']}

map = geemap.Map()
map.centerObject(polygon, zoom=15)
map.add_layer(rgb_case, rgb_vis, "orthophotomap")
map.add_layer(ndvi_case, ndvi_vis, "NDVI")
map.add_layer(bare_soil_case, bare_soil_vis, "bare soil mask")
map.add_layer(exceeded_case, exceeded_vis, "exceeded")
map.add_layer(polygon.style(**polygon_style), {}, "mining area")
map.add_layer(buildings_mask, {'min': 0, 'max': 1, 'palette': ['red', 'purple']}, "buildings")

map.add_layer(buildings.style(**buildings_style), {}, "Buildings (OSM)")

map

Map(center=[50.10339412565508, 19.63649860680545], controls=(WidgetControl(options=['position', 'transparent_b…

### TIME SERIES

In [4]:
actual_date_list = []
image_list = []

rectangle = polygon.geometry().buffer(500).bounds()

for start_date, end_date in zip(date_list_start, date_list_end):
    sentinel = (
        ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
        .filterBounds(polygon.geometry())
        .filterDate(start_date, end_date)
        .sort('CLOUDY_PIXEL_PERCENTAGE')
        .first()
    )
    
    if sentinel is not None:
        qa = sentinel.select('QA60').clip(rectangle)
        cloud_mask = qa.bitwiseAnd(1 << 10).neq(0)
        
        cloud_mask = cloud_mask.reduceRegion(
            reducer = ee.Reducer.mean(),
            geometry = rectangle,
            scale = 10,
            maxPixels = 1e9
        )

        cloud_cover = cloud_mask.get('QA60')
        
        if cloud_cover.getInfo() * 100 <= 0:
            date = ee.Date(sentinel.get('system:time_start')).format('YYYY-MM-dd').getInfo()
            image_list.append(sentinel)
            actual_date_list.append(date)
        
actual_date_list

['2016-01-14',
 '2016-03-24',
 '2016-04-03',
 '2016-05-23',
 '2016-06-29',
 '2016-07-02',
 '2016-08-28',
 '2016-09-10',
 '2016-11-29',
 '2016-12-16',
 '2017-01-28',
 '2017-02-27',
 '2017-04-05',
 '2017-05-28',
 '2017-06-27',
 '2017-07-22',
 '2017-08-08',
 '2017-09-27',
 '2017-10-02',
 '2017-11-04',
 '2017-12-26',
 '2018-02-19',
 '2018-03-04',
 '2018-04-20',
 '2018-05-13',
 '2018-06-07',
 '2018-08-13',
 '2018-09-12',
 '2018-10-05',
 '2018-11-06',
 '2019-02-19',
 '2019-03-24',
 '2019-04-20',
 '2019-05-25',
 '2019-06-09',
 '2019-07-29',
 '2019-08-26',
 '2019-09-22',
 '2019-10-20',
 '2019-11-01',
 '2019-12-11',
 '2020-01-03',
 '2020-03-28',
 '2020-04-02',
 '2020-05-09',
 '2020-07-01',
 '2020-08-22',
 '2020-09-09',
 '2020-10-06',
 '2020-11-08',
 '2021-02-26',
 '2021-03-03',
 '2021-04-09',
 '2021-05-12',
 '2021-06-21',
 '2021-07-13',
 '2021-08-20',
 '2021-09-06',
 '2021-10-09',
 '2021-11-25',
 '2022-01-09',
 '2022-02-16',
 '2022-03-25',
 '2022-04-12',
 '2022-05-19',
 '2022-06-03',
 '2022-07-

In [6]:
bare_soil_list = []

for image in image_list:
    image = image.clip(rectangle)
    rgb = image.select(['B4', 'B3', 'B2'])
    ndvi = image.normalizedDifference(['B8', 'B4'])
    
    bare_soil = ndvi.lt(0.18)
    bare_soil_list.append(bare_soil)
    
exceeded_list = []

for bare_soil in bare_soil_list:
    ones = ee.Image.constant(1).clip(rectangle)
    polygon_mask = ee.Image.constant(1).clip(polygon)
    outside_mask = ones.where(polygon_mask.eq(1), 0)

    exceeded = bare_soil.updateMask(outside_mask)
    exceeded_list.append(exceeded)

In [12]:
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
rgb_vis = {'min': 0, 'max': 3000}
polygon_style = {"color": "red", "fillColor": "00000000", "width": 1}
bare_soil_vis = {'min': 0, 'max': 1, 'palette': ['black', 'white']}
exceeded_vis = {'min': 0, 'max': 1, 'palette': ['black', 'yellow']}
color_palette = ['red', 'blue', 'green', 'yellow', 'purple']

map = geemap.Map()
map.centerObject(polygon, zoom=15)
map.add_layer(rgb, rgb_vis, "orthophotomap")
map.add_layer(ndvi, ndvi_vis, "NDVI")
map.add_layer(exceeded_list[13], exceeded_vis, f"exceeded {actual_date_list[13]}")
    
map.add_layer(polygon.style(**polygon_style), {}, "mining area")
map


Map(center=[50.10339412565508, 19.63649860680545], controls=(WidgetControl(options=['position', 'transparent_b…